<a href="https://colab.research.google.com/github/alexchen1999/covid-19-sample-strain-classification/blob/main/random_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Bio
!pip install bcbio-gff
!pip install ncbi-acc-download

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 5.1 MB/s 
     |████████████████████████████████| 2.3 MB 42.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.3 MB/s 
  Created wheel for bcbio-gff: filename=bcbio_gff-0.6.9-py3-none-any.whl size=15954 sha256=e1dbda75fb121b602febbec0e1ac3d6eb7c87b13aab68cd5484a4bbe570478be
  Stored in directory: /root/.cache/pip/wheels/fb/fc/e0/2e7658046d32b794fdfd1e0ec266dc8bae21c1811b18951b6d
Successfully built bcbio-gff
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import json
import numpy as np
import pandas as pd
import subprocess

from BCBio import GFF
from Bio import Align, SeqIO
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle

In [21]:
covid_strains = ['B.1.1.7_sequences.csv', 'B.1.617.2_sequences.csv', 'BA.1.1_sequences.csv']

In [22]:
alpha = pd.read_csv(covid_strains[0])
delta = pd.read_csv(covid_strains[1])
omicron = pd.read_csv(covid_strains[2])

In [23]:
shortest = min([alpha.shape[0], delta.shape[0], omicron.shape[0]])
print(shortest)

3803


In [24]:
alpha_sample = alpha.sample(n=shortest)
print(alpha_sample.shape[0])
delta_sample = delta.sample(n=shortest)
print(delta_sample.shape[0])
omicron_sample = omicron.sample(n=shortest)
print(omicron_sample.shape[0])

3803
3803
3803


In [25]:
print(alpha_sample['Country'].unique())
print(delta_sample['Country'].unique())
print(omicron_sample['Country'].unique())

['USA' 'Spain' 'Japan' 'Nigeria' 'Bangladesh' 'Austria' 'India' 'Iraq'
 'Djibouti' 'West Bank' 'Taiwan']
['USA' 'Egypt' 'Bangladesh' 'Mongolia' 'Bahrain' 'Uzbekistan' 'Pakistan'
 'Myanmar' 'India' 'Chile' 'Denmark' 'China' 'Gabon' 'West Bank' 'Japan'
 'Jamaica' 'Russia']
['USA' 'Bahrain' 'Japan' 'France' 'Pakistan']


In [26]:
# From SARS-Cov-2 GFF file, find genomic location of Spike Glycoprotein
in_file = "GCF_009858895.2_ASM985889v3_genomic.gff"
in_handle = open(in_file)
features = []
for rec in GFF.parse(in_handle):
    features = rec.features
in_handle.close()

start = -1
end = -1

# spike glycoprotein locus tag GU280_gp02
for i in range(len(features)):
    if "GU280_gp02" in features[i].id:
      print(features[i])
      print(features[i].location)
      print(int(features[i].location._start))
      print(int(features[i].location._end))
      start = int(features[i].location._start)
      end = int(features[i].location._end)

type: gene
location: [21562:25384](+)
id: gene-GU280_gp02
qualifiers:
    Key: Dbxref, Value: ['GeneID:43740568']
    Key: ID, Value: ['gene-GU280_gp02']
    Key: Name, Value: ['S']
    Key: gbkey, Value: ['Gene']
    Key: gene, Value: ['S']
    Key: gene_biotype, Value: ['protein_coding']
    Key: gene_synonym, Value: ['spike glycoprotein']
    Key: locus_tag, Value: ['GU280_gp02']
    Key: source, Value: ['RefSeq']

[21562:25384](+)
21562
25384


In [27]:
# Find spike glycoprotein sequence from reference genome
record = SeqIO.read("NC_045512.2.fa", "fasta")

# Take from 21562 to 25384
spike_seq = str(record._seq)[start:end]
print(spike_seq)
print(len(spike_seq))

ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTGTTAATCTTACAACCAGAACTCAATTACCCCCTGCATACACTAATTCTTTCACACGTGGTGTTTATTACCCTGACAAAGTTTTCAGATCCTCAGTTTTACATTCAACTCAGGACTTGTTCTTACCTTTCTTTTCCAATGTTACTTGGTTCCATGCTATACATGTCTCTGGGACCAATGGTACTAAGAGGTTTGATAACCCTGTCCTACCATTTAATGATGGTGTTTATTTTGCTTCCACTGAGAAGTCTAACATAATAAGAGGCTGGATTTTTGGTACTACTTTAGATTCGAAGACCCAGTCCCTACTTATTGTTAATAACGCTACTAATGTTGTTATTAAAGTCTGTGAATTTCAATTTTGTAATGATCCATTTTTGGGTGTTTATTACCACAAAAACAACAAAAGTTGGATGGAAAGTGAGTTCAGAGTTTATTCTAGTGCGAATAATTGCACTTTTGAATATGTCTCTCAGCCTTTTCTTATGGACCTTGAAGGAAAACAGGGTAATTTCAAAAATCTTAGGGAATTTGTGTTTAAGAATATTGATGGTTATTTTAAAATATATTCTAAGCACACGCCTATTAATTTAGTGCGTGATCTCCCTCAGGGTTTTTCGGCTTTAGAACCATTGGTAGATTTGCCAATAGGTATTAACATCACTAGGTTTCAAACTTTACTTGCTTTACATAGAAGTTATTTGACTCCTGGTGATTCTTCTTCAGGTTGGACAGCTGGTGCTGCAGCTTATTATGTGGGTTATCTTCAACCTAGGACTTTTCTATTAAAATATAATGAAAATGGAACCATTACAGATGCTGTAGACTGTGCACTTGACCCTCTCTCAGAAACAAAGTGTACGTTGAAATCCTTCACTGTAGAAAAAGGAATCTATCAAACTTCTAACTTTAGAGTCCAACCAACAGAATCTATTGTTAGATTTCCTAATATTACAA

In [28]:
alpha_sample_accessions = []
for i in range(shortest):
  alpha_sample_accessions.append(alpha_sample.iloc[i]['Accession'])

In [ ]:
!ncbi-acc-download --help

usage: ncbi-acc-download [-h] [-m {nucleotide,protein}] [--api-key API_KEY]
                         [-e {none,loads,all,correct}]
                         [-F {fasta,genbank,featuretable,gff3}] [-o OUT]
                         [-p PREFIX] [-g RANGE] [-r] [--url] [-v]
                         NCBI-accession [NCBI-accession ...]

positional arguments:
  NCBI-accession

optional arguments:
  -h, --help            show this help message and exit
  -m {nucleotide,protein}, --molecule {nucleotide,protein}
                        Molecule type to download. Default: nucleotide
  --api-key API_KEY     Specify USER NCBI API key. More info at
                        https://www.ncbi.nlm.nih.gov/books/NBK25497/
  -e {none,loads,all,correct}, --extended-validation {none,loads,all,correct}
                        Perform extended validation. Possible options are
                        'none' to skip validation, 'loads' to check if the
                        sequence file loads in Biopython, or '

In [ ]:
# Test NCBI accession download
test = alpha_sample_accessions[0]
print(test)

MZ164127.1


In [ ]:
!ncbi-acc-download --format fasta $test

In [ ]:
for i in range(3):
  subprocess.run(["ncbi-acc-download", "--format", "fasta", alpha_sample_accessions[i]])

In [ ]:
# SeqIO
record = SeqIO.read("MZ039190.1.fa", "fasta")
seq = str(record._seq)[start:end]

FileNotFoundError: ignored

In [ ]:
# Pairwise alignment
aligner = Align.PairwiseAligner()
aligner.mode = 'global'
alignments = aligner.align(seq, spike_seq)

NameError: ignored

In [29]:
# Alpha sample accessions
# Take from 21562 to 25384 from samples, should be a good enough approximation
# Credit to https://2-bitbio.com/2018/06/one-hot-encode-dna-sequence-using.html

alpha_spike_seq_one_hot = pd.DataFrame(columns=["Accession", "Seq"])
alpha_spike_seq_df = pd.DataFrame(columns = ['Accession', 'Seq'])
for i in range(3):
  accession = alpha_sample_accessions[i]
  subprocess.run(["ncbi-acc-download", "--format", "fasta", accession])
  record = SeqIO.read(accession + ".fa", "fasta")
  spike_seq = str(record._seq)[start:end]

  # One-hot encoding
  seq_array = array(list(spike_seq))
  label_encoder = LabelEncoder()
  integer_encoded_seq = label_encoder.fit_transform(seq_array)
    
  # one hot the sequence
  onehot_encoder = OneHotEncoder(sparse=False)
  # reshape because that's what OneHotEncoder likes
  integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
  onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)

  # convert the sequence from ndarray to json string to simplify conversion
  onehot_encoded_seq = json.dumps(onehot_encoded_seq.tolist())

  alpha_spike_seq_df.loc[len(alpha_spike_seq_df.index)] = [accession, onehot_encoded_seq]
  subprocess.run(['rm', accession + ".fa"])

In [10]:
# All
dfs = [alpha_sample, delta_sample, omicron_sample]
labels = ['Alpha', 'Delta', 'Omicron']
spike_seq_df = pd.DataFrame(columns = ['Accession', 'Seq', 'Label'])
num_samples = 1000
for i in range(num_samples):
  for j in range(len(labels)):
    accession = dfs[j].iloc[i]['Accession']

    subprocess.run(["ncbi-acc-download", "--format", "fasta", accession])
    record = SeqIO.read(accession + ".fa", "fasta")
    spike_seq = str(record._seq)[start:end]

    # One-hot encoding
    seq_array = array(list(spike_seq))
    label_encoder = LabelEncoder()
    integer_encoded_seq = label_encoder.fit_transform(seq_array)
    
    #one hot the sequence
    onehot_encoder = OneHotEncoder(sparse=False)
    #reshape because that's what OneHotEncoder likes
    integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
    onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)

    # convert the sequence from ndarray to json string to simplify conversion
    onehot_encoded_seq = json.dumps(onehot_encoded_seq.tolist())

    spike_seq_df.loc[len(spike_seq_df.index)] = [accession, onehot_encoded_seq, labels[j]]
    subprocess.run(['rm', accession + ".fa"])
  print((i+1) / num_samples * 100, "% complete")

# df = shuffle(spike_seq_df)
# df.to_csv('spike_seq_one_hots.csv', index=False)

0.1 % complete
0.2 % complete
0.3 % complete
0.4 % complete
0.5 % complete


KeyboardInterrupt: ignored

In [31]:
# All - 100 samples
dfs = [alpha_sample, delta_sample, omicron_sample]
labels = ['Alpha', 'Delta', 'Omicron']
spike_seq_df = pd.DataFrame(columns = ['Accession', 'Seq', 'Label'])
num_samples = 20
for i in range(num_samples):
  for j in range(len(labels)):
    accession = dfs[j].iloc[i]['Accession']

    subprocess.run(["ncbi-acc-download", "--format", "fasta", accession])
    record = SeqIO.read(accession + ".fa", "fasta")
    spike_seq = str(record._seq)[start:end]

    # One-hot encoding
    seq_array = array(list(spike_seq))
    label_encoder = LabelEncoder()
    integer_encoded_seq = label_encoder.fit_transform(seq_array)
    
    #one hot the sequence
    onehot_encoder = OneHotEncoder(sparse=False)
    #reshape because that's what OneHotEncoder likes
    integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
    onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)

    # convert the sequence from ndarray to json string to simplify conversion
    onehot_encoded_seq = json.dumps(onehot_encoded_seq.tolist())

    spike_seq_df.loc[len(spike_seq_df.index)] = [accession, onehot_encoded_seq, labels[j]]
    subprocess.run(['rm', accession + ".fa"])
  print((i+1) / num_samples * 100, "% complete")

# df = shuffle(spike_seq_df)
# df.to_csv('spike_seq_one_hots_100.csv', index=False)

5.0 % complete
10.0 % complete
15.0 % complete
20.0 % complete
25.0 % complete
30.0 % complete
35.0 % complete
40.0 % complete
45.0 % complete
50.0 % complete
55.00000000000001 % complete
60.0 % complete
65.0 % complete
70.0 % complete
75.0 % complete
80.0 % complete
85.0 % complete
90.0 % complete
95.0 % complete
100.0 % complete


# Challenge: Fasta seqs have 'N' --> we need to one-hots encode this

In [33]:
df = spike_seq_df

# df = shuffle(df)
print(df['Seq'])

print(df['Seq'].shape)

0     [[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...
1     [[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...
2     [[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...
3     [[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...
4     [[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [...
5     [[0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...
6     [[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...
7     [[0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...
8     [[1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0....
9     [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [...
10    [[0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...
11    [[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [...
12    [[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...
13    [[0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...
14    [[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...
15    [[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...
16    [[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...
17    [[1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0

In [39]:
print(df.loc[17])
print(df.loc[17]['Seq'])
print(df.loc[44])
print(df.loc[50])

Accession                                           OM701027.1
Seq          [[1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0....
Label                                                  Omicron
Name: 17, dtype: object
[[1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 1.0], [1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 1.0], [1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0], [0.

In [37]:
subprocess.run(["ncbi-acc-download", "--format", "fasta", 'OM701027.1'])
record = SeqIO.read("OM701027.1.fa", "fasta")
seq = str(record._seq)[start:end]
print(seq)

ATCTTACAACCAGAACTCAATTACCCCCTGCATACACTAATTCTTTCACACGTGGTGTTTATTACCCTGACAAAGTTTTCAGATCCTCAGTTTTACATTCAACTCAGGACTTGTTCTTACCTTTCTTTTCCAATGTTACTTGGTTCCATGTTATCTCTGGGACCAATGGTACTAAGAGGTTTGATAACCCTGTCCTACCATTTAATGATGGTGTTTATTTTGCTTCCATTGAGAAGTCTAACATAATAAGAGGCTGGATTTTTGGTACTACTTTAGATTCGAAGACCCAGTCCCTACTTATTGTTAATAACGCTACTAATGTTGTTATTAAAGTCTGTGAATTTCAATTTTGTAATGATCCATTTTTGGACCACAAAAACAACAAAAGTTGGATGGAAAGTGAGTTCAGAGTTTATTCTAGTGCGAATAATTGCACTTTTGAATATGTCTCTCAGCCTTTTCTTATGGACCTTGAAGGAAAACAGGGTAATTTCAAAAATCTTAGGGAATTTGTGTTTAAGAATATTGATGGTTATTTTAAAATATATTCTAAGCACACGCCTATTATAGTGCGTNNNNNNNNNGATCTCCCTCAGGGTTTTTCGGCTTTAGAACCATTGGTAGATTTGCCAATAGGTATTAACATCACTAGGTTTCAAACTTTACTTGCTTTACNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNTTCAACCTAGGACTTTTCTATTAAAATATAATGAAAATGGAACCATTACAGATGCTGTAGACTGTGCACTTGACCCTCTCTCAGAAACAAAGTGTACGTTGAAATCCTTCACTGTAGAAAAAGGAATCTATCAAACTTCTAACTTTAGAGTCCAACCAACAGAATCTATTGTTAGATTTCCTAATATTACAAACTTGTGCCCTTTTGATGAAGTTTTTAACGCCACCAAATTTGCATCTGTTTATGCTTG

In [ ]:
l = np.array([1, 0, 0, 0])
print(l)
print(l.tolist())
print(json.dumps(l.tolist()))

l = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])
print(l)
print(l.tolist())
print(json.dumps(l.tolist()))

l = np.array([[1., 0., 0., 0.], [0., 1., 0., 0.]])
print(l)
print(l.tolist())
print(json.dumps(l.tolist()))

[1 0 0 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[[1 0 0 0]
 [0 1 0 0]]
[[1, 0, 0, 0], [0, 1, 0, 0]]
[[1, 0, 0, 0], [0, 1, 0, 0]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]]
[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0]]
[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0]]


In [ ]:
# Example ndarray conversion:

a = np.array([[0, 1, 0, 0], [1, 0, 0, 0]])
print(a)
print(type(a))
print(a.shape)
b = a.tolist()
s = json.dumps(b)
arr = np.array(json.loads(s))
print(arr)
print(type(arr))
print(arr.shape)

[[0 1 0 0]
 [1 0 0 0]]
<class 'numpy.ndarray'>
(2, 4)
[[0 1 0 0]
 [1 0 0 0]]
<class 'numpy.ndarray'>
(2, 4)


In [ ]:
# Final check that df is well-formed
df
print(len(df.iloc[0]['Seq']))

3822
